## 대신증권 API cybos plus 사용하기

### 필요 모듈 import

In [13]:
import win32com.client
import pandas as pd
import time
import datetime

### cybos plus 연결 체크

In [14]:
 # 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()
else:
    print("PLUS가 정상적 연결됨")


PLUS가 정상적 연결됨


#### KOSPI 31에 해당하는 종목코드 찾기

In [15]:
kospi31_list = pd.read_csv("./data/kospi31_list.csv")
kospi31_list

,code,name
0,U182,kospi50_index
1,A030200,KT
2,A033780,KT&G
3,A003550,LG
4,A051900,LG생활건강
5,A032640,LG유플러스
6,A066570,LG전자
7,A051910,LG화학
8,A035420,NAVER
9,A005490,POSCO


In [16]:
def get_kospi50_stock_data(stockitems, start, end):
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
    
    # 날짜, 종목명, 시가, 고가, 저가, 종가
    column_dailychart = ['code', 'name', 'date', 'open', 'high', 'low', 'close',]
    
    stockitems_dailychart = pd.DataFrame()
    
    s_year, e_year = start[:4], end[:4]
    s_day, e_day = start[4:], end[4:]
    
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(5, (0, 2, 3, 4, 5))
    instStockChart.SetInputValue(6, ord('D'))
    instStockChart.SetInputValue(9, ord('1'))

    for idx, stockitem in stockitems.iterrows():
        dailychart = pd.DataFrame()
        row = list(range(len(column_dailychart)))
        rows = list()
        
        instStockChart.SetInputValue(0, stockitem['code'])
        print(stockitem['name'], "주식 데이터를 가져옵니다.")
        
        y = int(s_year)
        while y <= int(e_year):
            # 시작일, 종료일 계산
            if s_year == y:
                s_y, s_d = s_year, s_day
            else:
                s_y, s_d = str(y), '0101'
                
            if e_year == y:
                e_y, e_d = e_year, e_day
            else:
                e_y, e_d = str(y), '1231'
                
            print(s_y + s_d + '~' + e_y + e_d + '의 데이터를 가져옵니다.')
            instStockChart.SetInputValue(3, s_y + s_d)
            instStockChart.SetInputValue(2, e_y + e_d)
            
            # 남은요청 있는지 확인
            remain_request_count = nCpCybos.GetLimitRemainCount(1)
            print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)

            if remain_request_count == 0:
                print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')
                while True:
                    time.sleep(2)
                    remain_request_count = nCpCybos.GetLimitRemainCount(1)

                    if remain_request_count > 0:
                        print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                        break
                    print('대기 중...')
                    
            # BlockRequest
            instStockChart.BlockRequest()

            # GetHeaderValue
            numData = instStockChart.GetHeaderValue(3)
            numField = instStockChart.GetHeaderValue(1)
            # GetDataValue
            print(numData)
            for i in range(numData):
                row[0] = stockitem['code']
                row[1] = stockitem['name']
                row[2] = instStockChart.GetDataValue(0, i)# 날짜
                row[3] = instStockChart.GetDataValue(1, i) # 시가
                row[4] = instStockChart.GetDataValue(2, i) # 고가
                row[5] = instStockChart.GetDataValue(3, i) # 저가
                row[6] = instStockChart.GetDataValue(4, i) # 종가
                rows.append(list(row))
            df =  pd.DataFrame(data = rows, columns= column_dailychart)
            dailychart = pd.concat([dailychart, df])
            y += 1    
        dailychart = dailychart.sort_values("date")
        dailychart = dailychart.drop_duplicates(['date'])
        dailychart = dailychart.reset_index(drop=True)
        stockitems_dailychart = pd.concat([stockitems_dailychart, dailychart])
    return stockitems_dailychart

In [18]:
market = kospi31_list[kospi31_list.code == "U182"]
kospi50index_data = get_kospi50_stock_data(market, "19800101", "20210914")
kospi50index_data

kospi50_index 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  18
0
19810101~19811231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  17
0
19820101~19821231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  16
0
19830101~19831231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  15
0
19840101~19841231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  14
0
19850101~19851231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  13
0
19860101~19861231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  12
0
19870101~19871231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  11
0
19880101~19881231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  10
0
19890101~19891231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  9
0
19900101~19901231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  8
0
19910101~19911231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  7
0
19920101~19921231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  6
0
19930101~19931231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  5
0
19940101~19941231의 데이터를 가져옵니다.
U182 kospi50_index 남은 요청 :  4
0
19950101~19951231

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.739990,833.580017,786.229980,826.950012
1,U182,kospi50_index,20000303,843.460022,845.299988,816.580017,832.000000
2,U182,kospi50_index,20000306,844.330017,849.239990,831.119995,836.200012
3,U182,kospi50_index,20000307,831.460022,833.369995,816.719971,833.349976
4,U182,kospi50_index,20000308,823.349976,847.530029,810.869995,844.239990
...,...,...,...,...,...,...,...
5316,U182,kospi50_index,20210909,2846.560059,2858.580078,2810.719971,2815.219971
5317,U182,kospi50_index,20210910,2818.320068,2830.219971,2805.340088,2821.600098
5318,U182,kospi50_index,20210913,2810.709961,2834.520020,2804.149902,2825.580078
5319,U182,kospi50_index,20210914,2832.570068,2859.149902,2831.300049,2842.000000


In [19]:
stocks = kospi31_list[kospi31_list.code != "U182"]
kospi31_data = get_kospi50_stock_data(stocks, "19800101", "20210914")
kospi31_data

KT 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  36
0
19810101~19811231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  35
0
19820101~19821231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  34
0
19830101~19831231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  33
0
19840101~19841231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  32
0
19850101~19851231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  31
0
19860101~19861231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  30
0
19870101~19871231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  29
0
19880101~19881231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  28
0
19890101~19891231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  27
0
19900101~19901231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  26
0
19910101~19911231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  25
0
19920101~19921231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  24
0
19930101~19931231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  23
0
19940101~19941231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  22
0
19950101~19951231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  21
0
19960101~19961231의 데이터를 가져옵니다.
A030200 KT 남은 요청 :  20
0
19970101~19971231의 데이터를 가져옵니다.

A051900 LG생활건강 남은 요청 :  17
0
19940101~19941231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  16
0
19950101~19951231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  15
0
19960101~19961231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  14
0
19970101~19971231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  13
0
19980101~19981231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  12
0
19990101~19991231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  11
0
20000101~20001231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  10
0
20010101~20011231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  9
170
20020101~20021231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  8
244
20030101~20031231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  7
247
20040101~20041231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  6
249
20050101~20051231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  5
249
20060101~20061231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  4
247
20070101~20071231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  3
246
20080101~20081231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  2
248
20090101~20091231의 데이터를 가져옵니다.
A051900 LG생활건강 남은 요청 :  1
253
20

대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
0
20010101~20011231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  59
170
20020101~20021231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  58
244
20030101~20031231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  57
247
20040101~20041231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  56
249
20050101~20051231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  55
249
20060101~20061231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  54
247
20070101~20071231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  53
246
20080101~20081231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  52
248
20090101~20091231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  51
253
20100101~20101231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  50
251
20110101~20111231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  49
248
20120101~20121231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  48
248
20130101~20131231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  47
247
20140101~20141231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  46
245
20150101~20151231의 데이터를 가져옵니다.
A051910 LG화학 남은 요청 :  45
248
20160101~20161231의 데이터를 가져

A010950 S-Oil 남은 요청 :  48
247
20070101~20071231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  47
246
20080101~20081231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  46
248
20090101~20091231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  45
253
20100101~20101231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  44
251
20110101~20111231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  43
248
20120101~20121231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  42
248
20130101~20131231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  41
247
20140101~20141231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  40
245
20150101~20151231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  39
248
20160101~20161231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  38
246
20170101~20171231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  37
243
20180101~20181231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  36
244
20190101~20191231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  35
246
20200101~20201231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  34
248
20210101~20211231의 데이터를 가져옵니다.
A010950 S-Oil 남은 요청 :  33
177
SK텔레콤 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A01

20140101~20141231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  34
245
20150101~20151231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  33
248
20160101~20161231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  32
246
20170101~20171231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  31
243
20180101~20181231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  30
244
20190101~20191231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  29
246
20200101~20201231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  28
248
20210101~20211231의 데이터를 가져옵니다.
A035250 강원랜드 남은 요청 :  27
177
고려아연 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  26
0
19810101~19811231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  25
0
19820101~19821231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  24
0
19830101~19831231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  23
0
19840101~19841231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  22
0
19850101~19851231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  21
0
19860101~19861231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  20
0
19870101~19871231의 데이터를 가져옵니다.
A010130 고려아연 남은 요청 :  19
0
19880101~19881231의 데이터를 가져옵니다.
A0101

...

19920101~19921231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  2
294
19930101~19931231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  1
296
19940101~19941231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
작업을 재개합니다. (남은 요청 : 60)
296
19950101~19951231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  59
293
19960101~19961231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  58
293
19970101~19971231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  57
292
19980101~19981231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  56
292
19990101~19991231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  55
249
20000101~20001231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  54
241
20010101~20011231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  53
246
20020101~20021231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  52
244
20030101~20031231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  51
247
20040101~20041231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  50
249
20050101~20051231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  49
249
20060101~20061231의 데이터를 가져옵니다.
A005930 삼성전자 남은 요청 :  48
247
20070101~2

114
20010101~20011231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  47
246
20020101~20021231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  46
244
20030101~20031231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  45
247
20040101~20041231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  44
249
20050101~20051231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  43
249
20060101~20061231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  42
247
20070101~20071231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  41
246
20080101~20081231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  40
248
20090101~20091231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  39
253
20100101~20101231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  38
251
20110101~20111231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  37
248
20120101~20121231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  36
248
20130101~20131231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  35
247
20140101~20141231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  34
245
20150101~20151231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  33
248
20160101~20161231의 데이터를 가져옵니다.
A036570 엔씨소프트 남은 요청 :  32
246
20170101~20171231의 데

20120101~20121231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  30
248
20130101~20131231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  29
247
20140101~20141231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  28
245
20150101~20151231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  27
248
20160101~20161231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  26
246
20170101~20171231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  25
243
20180101~20181231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  24
244
20190101~20191231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  23
246
20200101~20201231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  22
248
20210101~20211231의 데이터를 가져옵니다.
A003670 포스코케미칼 남은 요청 :  21
177
한국전력 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  20
0
19810101~19811231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  19
0
19820101~19821231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  18
0
19830101~19831231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  17
0
19840101~19841231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  16
0
19850101~19851231의 데이터를 가져옵니다.
A015760 한국전력 남은 요청 :  15
0
19860101~198

A018880 한온시스템 남은 요청 :  18
244
20190101~20191231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  17
246
20200101~20201231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  16
248
20210101~20211231의 데이터를 가져옵니다.
A018880 한온시스템 남은 요청 :  15
177
현대모비스 주식 데이터를 가져옵니다.
19800101~19801231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  14
0
19810101~19811231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  13
0
19820101~19821231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  12
0
19830101~19831231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  11
0
19840101~19841231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  10
0
19850101~19851231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  9
0
19860101~19861231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  8
0
19870101~19871231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  7
0
19880101~19881231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  6
0
19890101~19891231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  5
90
19900101~19901231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  4
291
19910101~19911231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  3
292
19920101~19921231의 데이터를 가져옵니다.
A012330 현대모비스 남은 요청 :  2
294


,code,name,date,open,high,low,close
0,A030200,KT,19981223,28750,28750,28750,28750
1,A030200,KT,19981224,33050,33050,33050,33050
2,A030200,KT,19981228,38000,38000,38000,38000
3,A030200,KT,19990104,43700,43700,42000,43700
4,A030200,KT,19990105,43500,44450,42500,43600
...,...,...,...,...,...,...,...
11170,A005380,현대차,20210909,212500,212500,208000,208500
11171,A005380,현대차,20210910,208500,209500,204000,206500
11172,A005380,현대차,20210913,206500,206500,204000,205000
11173,A005380,현대차,20210914,206500,211500,206000,209000


#### KOSPI index와 개별 주식 데이터 합치고 저장하기

In [20]:
kospi31_index_stocks = pd.concat([kospi50index_data, kospi31_data])
kospi31_index_stocks

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.74,833.58,786.23,826.95
1,U182,kospi50_index,20000303,843.46,845.3,816.58,832
2,U182,kospi50_index,20000306,844.33,849.24,831.12,836.2
3,U182,kospi50_index,20000307,831.46,833.37,816.72,833.35
4,U182,kospi50_index,20000308,823.35,847.53,810.87,844.24
...,...,...,...,...,...,...,...
11170,A005380,현대차,20210909,212500,212500,208000,208500
11171,A005380,현대차,20210910,208500,209500,204000,206500
11172,A005380,현대차,20210913,206500,206500,204000,205000
11173,A005380,현대차,20210914,206500,211500,206000,209000


In [22]:
kospi31_index_stocks.to_csv("./data/kospi31-2003-2021.csv", index=False)